### Coffee Bean Identifier using Neural Networks

This notebook would serve as a consolidated file of the different resources utilized within this project. While this has been proved to be working and functioning, it is important to take note that the current state of this model is hence, only for **Proof of Concept purposes**. Further Exploratory Data Analysis, Feature Engineering, and other data activities are advised to achieve optimal and accurate performance.

# The Dataset
 - 

During the creation of this model, there are various neural network models used to perform the identification of data. These are
#### > VGG19 <br>
![VGG 19 Architecture](vgg-19.png)
 - It is a deep convolutional neural network architecture proposed by the Visual Geometry Group at the University of Oxford. VGG19 consists of 19 layers, including 16 convolutional layers and 3 fully connected layers. The network is known for its simplicity and uniformity, where each layer in the network uses a small 3x3 convolutional kernel and 2x2 max-pooling layers to process the input images. VGG19 has been widely used as a benchmark for image recognition tasks due to its straightforward architecture.
#### > ResNet50 <br>
![Resnet50 Architecture](resnet50.ppm)
 - ResNet50, short for Residual Network 50, is a variant of the ResNet architecture introduced by Microsoft Research. ResNet is based on the concept of residual learning, which tackles the vanishing gradient problem in very deep neural networks. ResNet50 specifically refers to a ResNet with 50 layers. The network introduces skip connections, also known as shortcut connections, that enable the network to learn the residual between the input and output of each layer, making it easier to train extremely deep networks. ResNet50 is a popular choice for image classification and other computer vision tasks due to its effectiveness in handling deeper architectures.
#### > InceptionV3 <br>
![Inceptionv3 Architecture](inceptionv3.jpg)
 - a deep convolutional neural network architecture developed by Google's DeepMind team. It is an evolution of the original Inception architecture (also known as GoogLeNet). InceptionV3 is designed to improve computational efficiency while maintaining high accuracy in image recognition tasks. It achieves this by using a combination of 1x1, 3x3, and 5x5 convolutions in parallel to capture features at different scales. Additionally, it incorporates the concept of factorizing convolutions to reduce the number of parameters and computations required. InceptionV3 has been widely used in various applications and is known for its competitive performance on image classification tasks.
#### > MobileNetV2 <br>
![MobileNetv2 Architecture](mobilenetv2.png)
 - MobileNetV2 is a lightweight deep neural network architecture developed by Google. It is designed for efficient deployment on mobile and embedded devices, where computational resources are limited. MobileNetV2 achieves efficiency by employing depthwise separable convolutions, which split the standard convolution operation into separate depthwise and pointwise convolutions, reducing the number of computations required. This allows MobileNetV2 to be much faster and smaller in size compared to traditional architectures while maintaining reasonable accuracy on image classification tasks. Due to its efficiency, MobileNetV2 has found applications in various mobile and real-time vision-based applications.

## The Training Process

In [ ]:
# Import required libraries
# import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV2


class ImageClassifierTrainer:
    def __init__(
        self,
        model_name,
        img_size=224,
        num_classes=4,
        train_dir="../dataset/train",
        val_dir="../dataset/validation",
    ):
        # Define the input size and number of classes
        self.img_size = img_size
        self.num_classes = num_classes

        # Specify the pre-trained model to use
        if model_name == "VGG19":
            self.pretrained_model = VGG19(
                weights="imagenet",
                include_top=False,
                input_shape=(img_size, img_size, 3),
            )
        elif model_name == "ResNet50":
            self.pretrained_model = ResNet50(
                weights="imagenet",
                include_top=False,
                input_shape=(img_size, img_size, 3),
            )
        elif model_name == "InceptionV3":
            self.pretrained_model = InceptionV3(
                weights="imagenet",
                include_top=False,
                input_shape=(img_size, img_size, 3),
            )
        elif model_name == "MobileNetV2":
            self.pretrained_model = MobileNetV2(
                weights="imagenet",
                include_top=False,
                input_shape=(img_size, img_size, 3),
            )
        else:
            raise ValueError(
                "Invalid model name. " +
                "Supported models are" +
                "VGG19, ResNet50, InceptionV3, and MobileNetV2."
            )

        # Freeze the layers in the pre-trained model
        for layer in self.pretrained_model.layers:
            layer.trainable = False

        # Add a new classifier on top
        x = self.pretrained_model.output
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(512, activation="relu")(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        predictions = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

        # Define the new model
        self.model = tf.keras.models.Model(
            inputs=self.pretrained_model.input, outputs=predictions
        )

        # Compile the model
        self.model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"],
            run_eagerly=True
        )

        # Define the data generators for training and validation
        train_datagen = ImageDataGenerator(
            rescale=1.0 / 255,
            rotation_range=20,
            zoom_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
        )

        val_datagen = ImageDataGenerator(rescale=1.0 / 255)

        self.train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(img_size, img_size),
            batch_size=32,
            class_mode="categorical",
        )

        self.val_generator = val_datagen.flow_from_directory(
            val_dir,
            target_size=(img_size, img_size),
            batch_size=32,
            class_mode="categorical",
        )

    def train(self, epochs):
        # Train the model
        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.train_generator.samples//epochs,
            epochs=10,
            validation_data=self.val_generator,
            validation_steps=self.val_generator.samples//epochs,
        )

    def save_model(self, model_filename):
        # Save the trained model
        self.model.save(model_filename)


## The Trainer GUI

In [ ]:
import sys
import os
import re
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QTableView,
    QWidget,
    QVBoxLayout,
    QHBoxLayout,
    QLabel,
    QPushButton,
    QFileDialog,
    QComboBox,
    QPlainTextEdit,
    QToolBar,
    QToolButton,
    QMenu
)
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QStandardItemModel, QStandardItem, QTextCursor
import shutil

from trainer import ImageClassifierTrainer


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Create a table view
        self.table_view = QTableView(self)
        # self.setFixedSize(800, 600)
        self.setWindowFlags(
            Qt.Window
            | Qt.CustomizeWindowHint
            | Qt.WindowTitleHint
            | Qt.WindowCloseButtonHint
        )

        # Create a data model for the table
        self.model = QStandardItemModel(0, 2, self)
        self.model.setHorizontalHeaderLabels(["File Name", "Ripeness"])
        self.table_view.setModel(self.model)

        # Create an image preview widget
        self.image_label = QLabel(self)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.show_image("placeholder.jpg")

        # Create two buttons
        self.button1 = QPushButton("Add Images", self)
        self.button2 = QPushButton("Train Models", self)
        self.button1.clicked.connect(self.add_images)
        self.button2.clicked.connect(self.train_model)

        # Create a horizontal layout to hold the image and buttons
        image_layout = QVBoxLayout()
        image_layout.addWidget(self.image_label)
        image_layout.addWidget(self.button1)
        image_layout.addWidget(self.button2)

        # Create a horizontal layout to hold the table and image layouts
        table_widget = QWidget(self)
        layout = QHBoxLayout(table_widget)
        layout.addWidget(self.table_view)
        layout.addLayout(image_layout)

        # Set the central widget of the main window
        self.setCentralWidget(table_widget)

        # Connect the clicked signal of the table view to a custom slot
        self.table_view.clicked.connect(self.handle_table_click)

        self.text_edit = QPlainTextEdit()
        # Redirect terminal output to the log
        sys.stdout = self
        # Initialize the output buffer
        self.buffer = ""
        image_layout.addWidget(self.text_edit)
        self.setContextMenuPolicy(Qt.CustomContextMenu)
        self.customContextMenuRequested.connect(self.show_context_menu)
    
    def show_context_menu(self, position):
        context_menu = QMenu(self)
        action1 = context_menu.addAction("Delete")

        action1.triggered.connect(self.action1_triggered)

        context_menu.exec(self.table_view.viewport().mapToGlobal(position))

    def action1_triggered(self):
        index = self.table_view.currentIndex()
        if index.isValid():
            self.table_view.model().removeRow(index.row())

    def write(self, message):
        # Append the message to the output buffer
        self.buffer += message

        # If a newline character is encountered, flush the buffer to the log
        if "\n" in message:
            lines = self.buffer.split("\n")
            for line in lines[:-1]:
                # Write each line to the log
                line = re.sub(r"[^\x20-\x7E]+", "", line)
                self.text_edit.moveCursor(QTextCursor.End)
                self.text_edit.insertPlainText(line + "\n")
            # Clear the buffer
            self.buffer = lines[-1]

    def flush(self):
        pass

    def show_image(self, filename):
        pixmap = QPixmap(filename)
        pixmap = pixmap.scaledToHeight(200)
        self.image_label.setPixmap(pixmap)

    def add_images(self):
        files, _ = QFileDialog.getOpenFileNames(
            self, "Open file", "", "Images (*.png *.xpm *.jpg *.bmp *.gif)"
        )
        for i, file in enumerate(files):
            self.show_image(file)
            row = self.model.rowCount()
            self.model.insertRow(row)
            combo_box = QComboBox(self)
            combo_box.addItems(["Ripe", "Unripe", "Semi Ripe", "Overripe"])
            for column in range(1):
                self.model.setItem(row, 0, QStandardItem(file))
                self.table_view.setIndexWidget(self.model.index(row, 1), combo_box)

    def handle_table_click(self, index):
        # Get the selected row and column index
        index = self.table_view.selectedIndexes()[0]
        row = index.row()
        self.show_image(self.model.item(row, 0).text())

    def train_model(self):
        num_rows = self.model.rowCount()

        # Create the folders if they don't exist
        for folder in ["Ripe", "Unripe", "Semi Ripe", "Overripe"]:
            if not os.path.exists(f"../dataset/train/{folder}"):
                print(f"Creating {folder} folder for training")
                os.makedirs(f"../dataset/train/{folder}")
            if not os.path.exists(f"../dataset/validation/{folder}"):
                print(f"Creating {folder} folder for validation")
                os.makedirs(f"../dataset/validation/{folder}")

        for row in range(num_rows):
            # Get the combo box widget for the current row
            combo_box = self.table_view.indexWidget(self.model.index(row, 1))
            # Get the selected item from the combo box
            item_data = combo_box.currentText()
            print(f"{row}: {item_data} {self.model.item(row, 0).text()}")
            # get the file extension
            file_extension = self.model.item(row, 0).text().split(".")[-1]
            # Transfer the image to the appropriate folder
            shutil.copy(
                self.model.item(row, 0).text(),
                f"../dataset/train/{item_data}/image_{row}.{file_extension}",
            )
            print(f"copying to ../dataset/train/{item_data}/image_{row}.{file_extension}")
        # Train the model
        trainer = ImageClassifierTrainer("VGG19")
        trainer.train(num_rows)
        trainer.save_model("model.h5")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

## Testing Phase

In [ ]:
import numpy as np
import keras.utils as image
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.applications.vgg19 import VGG19


class VGG19CoffeeClassifier:
    def __init__(self, pretrained=True, model_path=None):
        if pretrained:
            # Load the pre-trained VGG19 model without the top layers
            self.base_model = VGG19(weights='imagenet', include_top=False)

            # Add new classification layers to the model
            x = self.base_model.output
            x = GlobalAveragePooling2D()(x)
            x = Dense(1024, activation='relu')(x)
            predictions = Dense(4, activation='softmax')(x)

            # Define the new model with the added classification layers
            self.model = Model(inputs=self.base_model.input, outputs=predictions)

            # Freeze the weights of the pre-trained layers
            for layer in self.base_model.layers:
                layer.trainable = False

            # Compile the model with a SGD optimizer and categorical crossentropy loss
            self.model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
        else:
            # Load the custom model from the .h5 file
            self.model = load_model(model_path)

        # Define the image size for the model input
        self.img_width, self.img_height = 224, 224

        # Define the label dictionary
        self.label_dict = {0: "Unripe", 1: "Semi-ripe", 2: "Ripe", 3: "Overripe"}

    def classify(self, image_path):
        # Load the image to classify
        img = image.load_img(image_path, target_size=(self.img_width, self.img_height))

        # Convert the image to an array and preprocess it for input into the model
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Make a prediction with the model
        preds = self.model.predict(x)

        # Decode the predictions and return the predicted label
        pred_index = np.argmax(preds)
        pred_label = self.label_dict[pred_index]
        accuracy = preds[0][pred_index]
        return {"accuracy": f"{accuracy * 100}%", "label": pred_label}


# Usage
# vgg_coffee = VGG19CoffeeClassifier(pretrained=True, model_path='my_model.h5')
# image_path = 'coffee_berry.png'
# pred_label = vgg_coffee.classify(image_path)
# print('Predicted:', pred_label)

## Working POC